In [1]:
import json
from pathlib import Path
import chromadb
from chromadb.utils.embedding_functions.sentence_transformer_embedding_function import (
    SentenceTransformerEmbeddingFunction,
)

from gliner import GLiNER
from pydantic import BaseModel

from chunking_utils import get_chunks
from llm_utils import ask_llm
from metadata_utils import get_meta
from nlp_utils import get_entities, get_tags, get_relevant_chunks
from transcript_utils import srt_to_text


class Entity(BaseModel):
    start: int
    end: int
    text: str
    label: str
    score: float


model = GLiNER.from_pretrained("urchade/gliner_base", max_length=768)

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

/home/ansel/ai/graphrag/.venv/lib/python3.12/site-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [2]:
chromadb_dir = Path("chromadb")
chromadb_dir.mkdir(exist_ok=True)

chroma_client = chromadb.PersistentClient(path=str(chromadb_dir))
collection = chroma_client.get_or_create_collection(name="keywords")

In [ ]:
files = [
    file for file in sorted(Path("../files/rotl").iterdir()) if ".srt" in file.name
]

LLM_MODEL = "qwen2.5:14b"

for file in files:
    file_name, episode_number, episode_date, episode_title = get_meta(file)
    transcript = srt_to_text(file)
    chunks = get_chunks(transcript)
    results = get_entities(chunks, model)

    for entity, data in results.items():
        labels = data["labels"]
        indexes = data["indexes"]
        relevant_chunks = get_relevant_chunks(chunks, indexes)

        context = "\n".join(relevant_chunks)
        question = f"What do John and Merlin say about {entity}?"
        answer = ask_llm(f"{context}\n\n{question}", model=LLM_MODEL, tokens=500)

        tags = get_tags(answer, model, stopwords=["john", "merlin"])

        doc = f"{entity}\n\n{", ".join(labels)}\n\n{", ".join(tags)}\n\n{answer}"

        id = f"{entity}_rotl_{episode_number}"
        metadata = {
            "chunks": ",".join([str(i) for i in indexes]),
            "show": "Roderick on the Line",
            "episode": episode_number,
            "title": episode_title,
            "subject": entity,
            "category": ",".join(labels),
            "tags": ",".join(tags),
        }

        collection.add(documents=[doc], ids=[id], metadatas=[metadata])

In [ ]:
query = "Who is Eleanor?"
results = collection.query(query_texts=[query], n_results=10)

docs = results["documents"][0]

answer = ask_llm(
    f"{query} Only answer the question.\n\n{docs[0]}", model=LLM_MODEL, tokens=500
)

print(answer)